In [42]:
import torch
import pandas as pd
import string
from tqdm import tqdm
import numpy as np
import pickle

# HyperParameter

In [43]:
EPOCH = 20
DATA_NUM = 10000

In [44]:
stop_words = []
with open('./NLPUtils/english.txt', encoding='utf-8') as f:
    stop_words = [stop_word.replace('\n','') for stop_word in f.readlines()]

In [45]:
file_path = './basic_dataset/IMDB/IMDB Dataset.csv'

df = pd.read_csv(file_path, encoding='utf-8')

In [46]:
sentences = [sentence.lower() for sentence in df['review'][:DATA_NUM]]

In [47]:
print(f'Raw text Data -> {sentences[0]}')

Raw text Data -> one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.<br /><br />the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.<br /><br />it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />i would say the main appeal of the show 

In [48]:
for i in tqdm(range(len(sentences))):
    sentences[i] = sentences[i].translate(str.maketrans('', '', string.punctuation))

100%|██████████| 10000/10000 [00:00<00:00, 69914.51it/s]


In [49]:
def tokenize(sentences) -> list:
    tokens = [sentence.split() for sentence in sentences]
    return tokens

In [50]:
tokens = tokenize(sentences)

In [51]:
for i in tqdm(range(len(tokens))):
    for stop_word in stop_words:
        tokens[i] = list(filter(stop_word.__ne__, tokens[i]))

100%|██████████| 10000/10000 [00:38<00:00, 256.84it/s]


In [52]:
print(f'Preprocess Data -> {tokens[0]}')

Preprocess Data -> ['reviewers', 'mentioned', 'watching', '1', 'oz', 'episode', 'youll', 'hooked', 'happened', 'mebr', 'br', 'struck', 'oz', 'brutality', 'unflinching', 'scenes', 'violence', 'set', 'word', 'trust', 'faint', 'hearted', 'timid', 'pulls', 'punches', 'drugs', 'sex', 'violence', 'hardcore', 'classic', 'wordbr', 'br', 'called', 'oz', 'nickname', 'oswald', 'maximum', 'security', 'penitentary', 'focuses', 'emerald', 'city', 'experimental', 'prison', 'cells', 'glass', 'fronts', 'face', 'inwards', 'privacy', 'high', 'agenda', 'em', 'city', 'manyaryans', 'muslims', 'gangstas', 'latinos', 'christians', 'italians', 'irish', 'moreso', 'scuffles', 'death', 'stares', 'dodgy', 'dealings', 'shady', 'agreements', 'awaybr', 'br', 'main', 'appeal', 'fact', 'wouldnt', 'forget', 'pretty', 'pictures', 'painted', 'mainstream', 'audiences', 'forget', 'charm', 'forget', 'romanceoz', 'doesnt', 'mess', 'episode', 'struck', 'nasty', 'surreal', 'ready', 'watched', 'developed', 'taste', 'oz', 'accust

In [53]:
voc = []

In [54]:
for sentence in tqdm(tokens):
     for word in sentence:
        if word not in voc:
            voc.append(word)

100%|██████████| 10000/10000 [01:26<00:00, 116.04it/s]


In [55]:
print(f'The size of voc {len(voc)}')
print(f'The first word of voc is {voc[0]}')

The size of voc 74274
The first word of voc is reviewers


In [56]:
with open('./basic_dataset/IMDB/voc.data', 'wb') as f:
    pickle.dump(voc, f)

In [57]:
word2idx = {w:idx for (idx, w) in enumerate(voc)}
inx2word = {idx:w for (idx, w) in enumerate(voc)}

In [58]:
window_size = 2
idx_pair = []
for sentence in tqdm(tokens):
    indices = [word2idx[word] for word in sentence]
    for idx in range(len(indices)):
        if idx + window_size - 1 < len(indices):
            window = indices[idx : idx + window_size]
            for x_word in window:
                for y_word in window:
                    if x_word != y_word:
                        idx_pair.append((x_word, y_word))

100%|██████████| 10000/10000 [00:00<00:00, 14161.09it/s]


In [59]:
with open('./basic_dataset/IMDB/XY_list.data', 'wb') as f:
    pickle.dump(idx_pair, f)